In [1]:
import json, requests
from folium import Map, TileLayer, GeoJson


In [2]:
endpoint = "http://127.0.0.1:8082"
geojson = {"type": "Feature", "geometry": {"coordinates": [[[-87.0251, 36.2251], [-87.0251, 36.0999], [-85.4249, 36.0999], [-85.4249, 36.2251], [-87.0251, 36.2251]]], "type": "Polygon"}}
bounds = (-87.0251, 36.0999, -85.4249, 36.2251)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)
geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

In [3]:
print (bounds)

(-87.0251, 36.0999, -85.4249, 36.2251)


In [4]:
body = {
    "collections": ["noaa-emergency-response"],
    "bbox": bounds,
}
response = requests.post(
    f"{endpoint}/mosaic/register",
    json=body,
).json()
print(response)

{'searchid': 'ae452624958df588858148afc25f6b3e', 'links': [{'rel': 'metadata', 'type': 'application/json', 'href': 'http://127.0.0.1:8082/mosaic/ae452624958df588858148afc25f6b3e/info'}, {'rel': 'tilejson', 'type': 'application/json', 'href': 'http://127.0.0.1:8082/mosaic/ae452624958df588858148afc25f6b3e/tilejson.json'}]}


In [5]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)
tj_resp = requests.get(
    response['links'][1]['href'],
    params={
        # Info to add to the tilejson response
        "minzoom": 13,
        "maxzoom": 20,
        # query parameter to add to the tile URL
        "assets": "cog",
    }
).json()
print(tj_resp)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=tj_resp["tiles"][0],
    attr="Mosaic",
    min_zoom=13,
    max_zoom=20,
    max_native_zoom=20,    
)
aod_layer.add_to(m)
m

{'tilejson': '2.2.0', 'name': 'ae452624958df588858148afc25f6b3e', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['http://127.0.0.1:8082/mosaic/ae452624958df588858148afc25f6b3e/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog'], 'minzoom': 13, 'maxzoom': 20, 'bounds': [-87.0251, 36.0999, -85.4249, 36.2251], 'center': [-86.225, 36.162499999999994, 13]}
